<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/leogabac-Lab3/Lab3/leogabac_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative Methods**
**Leonardo Gabriel Alanis Cantú**

# **Abstract**

Exact methods are really useful whenever they work, until they do not. Sometimes when dealing with massive amount of data, or really complex systems, considering the accuracy/performance ratio, it is better to deal with approximate solutions. Additionally, iterative methods help us to develop methods for those type of problems for which the solution cannot be computed exactly, an example of this is non-linear dynamics.
In this report, we will look at two approximation methods for systems of linear equations, the Jacobi approximation method and the Gauss-Seidel method. Additionally, we look at one of the most famous methods for root finding in one variable: the Newton's method in one variable.

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [ ]:
# Load neccessary modules.
#from google.colab import files

import time
import numpy as np
import math
import scipy as sp
from scipy import special

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

The _fixed point iteration_ methods are based on the following ansatz

$$
x^{(k+1)} = g(x^{(k)}),
$$

one of the easiest form is to assume a linear function, this is called _stationary iterative methods_

$$
x^{(k+1)} = Mx^{(k)} + c.
$$

When solving a system of linear equations $Ax = b$, if we consider the splitting

$$
A = \text{diag}(A) + \text{the rest}  =D + (A-D),
$$

and the iteration matrix, and vector

$$
M = (I - D^{-1}A), \quad c = D^{-1}b.
$$

Componen wise, the Jacobi iteration takes the form

$$
x_{i}^{(k+1)}=a_{i i}^{-1}\left(b_{i}-\sum_{j \neq i} a_{i j} x_{j}^{(k)}\right), \quad \forall i
$$

This method converges if $\| I -D^{-1}A \| < 1 $. The _Gauss-Seidel method_ is based on another fixed-point iteration with splitting

$$
A = L + (A-L),
$$

with the following iteration matrix and vector

$$
M_{G S}=-L^{-1}(A-L)=\left(I-L^{-1} A\right), \quad L^{-1}b,
$$

which in component wise form lets us have the form

$$
x_{i}^{(k+1)}=a_{i i}^{-1}\left(b_{i}-\sum_{j<i} a_{i j} x_{j}^{(k+1)}-\sum_{j>i} a_{i j} x_{j}^{(k)}\right), \quad \forall i .
$$
With convergence criterion $\| I -L^{-1}A \| < 1 $

Newton's method tries to solve the equation $f(x) = 0$ by Taylor expanding around the  $k$-th iteration

$$
0 = f(x) = f(x^{(k)}) + f'(x^{(k)})(x-x^{(k)}) + \cdots,
$$

and by isolating, we get

$$
x^{(k+1)} = x^{(k)} - [f'(x^{(k)})]^{-1}f(x^{(k)})
$$

Note that the method requires to evaluate the first derivative at a point. Ideally, we will not be provided with the derivative, and we want to compute everything numerically, hence a forward differenciation scheme will be implemented.

$$
f'(x) \approx \dfrac{f(x+h) - f(x)}{h}
$$

# **Method**

Jacobi iteration

In [ ]:
def jacobi_iteration(A,b,iter = 100):
    # ===== Input ===== #
    # A: n x n matrix non-singular matrix
    # b: n-vector
    # ===== Output ===== #
    # x: n-vector solution to Ax = b
    
    assert A.shape[0] == A.shape[1], "Not square matrix"
    assert A.shape[0] == len(b), "A and b Dimension mismatch"
    
    n = A.shape[0]
    x = np.zeros(n)
    xnew = np.zeros(n)
    
    for _ in range(iter):
        for i in range(n):
            s = sum([ A[i,j]*x[j] for j in range(n) if j!=i]) # A neat one-liner
            xnew[i] = (1/A[i,i])*(b[i] - s)
        x = np.copy(xnew)
    
    return x

Gauss-Seidel

In [ ]:
def gauss_seidel(A,x,iter = 100):
    # ===== Input ===== #
    # A: n x n matrix non-singular matrix
    # b: n-vector
    # ===== Output ===== #
    # x: n-vector solution to Ax = b
    
    assert A.shape[0] == A.shape[1], "Not square matrix"
    assert A.shape[0] == len(b), "A and b Dimension mismatch"
    
    n = A.shape[0]
    x = np.zeros(n)
    xnew = np.zeros(n)
    
    for _ in range(iter):
        for i in range(n):
            s1 = sum([ A[i,j]*xnew[j] for j in range(i)])
            s2 = sum([ A[i,j]*x[j] for j in range(i+1,n)])
            xnew[i] = (1/A[i,i])*( b[i] - s1 - s2)
    
        x = np.copy(xnew)
    return x
    

Newton's Method for roots

In [ ]:
def forward_diff(f,x,h):
    df = f(x+h)-f(x)
    return df/h

def newton_univar(f,x0,h = 0.01,iter = 10):
    
    # ===== Input ===== #
    # f: A function for which we want to compute its roots
    # x0: An initial guess of the root
    # h: The step size to use in the differentiation process
    # ===== Output ===== #
    # x: Approximate root of the function
    
    x = x0 # initialize
    
    for _ in range(iter):
        df = forward_diff(f,x,h)
        x -= f(x)/df
    return x    

# **Results**

We will test the Jacobi iteration method and Gauss-Seidel with the same two systems of equations, using only 5 iterations for each, so that we can compare accuracy between the two methods. If we were to compare with lots of iterations, there would not be too much room to compare, since they are expected to converge eventually.

Additionally, we will compare the residual error with the exact solutions obtained by Wolfram Mathematica.

For the Jacobi method:

In [ ]:
A = np.array([[9,2,3], [1,12,9], [4,6,14] ] )
b = np.array([7,2,1])
y = np.array([95/118,19/59,-35/118])
x = jacobi_iteration(A,b,iter = 5)
print("|Ax - b| =",np.linalg.norm(A@x - b) )
print("|x - y| =",np.linalg.norm(x - y) )

|Ax - b| = 0.19982519370867116
|x - y| = 0.010491991240013399


In [ ]:
A = np.array([[5,-1,2], [3,8,-2], [1,1,4] ] )
b = np.array([12,-25,6])
y = np.array([1,-3,2])
x = jacobi_iteration(A,b,iter = 5)
print("|Ax - b| =",np.linalg.norm(A@x - b) )
print("|x - y| =",np.linalg.norm(x - y) )

|Ax - b| = 0.27185263832472084
|x - y| = 0.04058281331966143


Whereas for the Gauss-Seidel

In [ ]:
A = np.array([[9,2,3], [1,12,9], [4,6,14] ] )
b = np.array([7,2,1])
y = np.array([95/118,19/59,-35/118])
x = gauss_seidel(A,b, iter = 5)
print("|Ax - b| =",np.linalg.norm(A@x - b) )
print("|x - y| =",np.linalg.norm(x - y) )

|Ax - b| = 0.013255355864182633
|x - y| = 0.001602593491056047


In [ ]:
A = np.array([[5,-1,2], [3,8,-2], [1,1,4] ] )
b = np.array([12,-25,6])
y = np.array([1,-3,2])
x = gauss_seidel(A,b, iter= 5)
print("|Ax - b| =",np.linalg.norm(A@x - b) )
print("|x - y| =",np.linalg.norm(x - y) )

|Ax - b| = 0.0008591570135093311
|x - y| = 0.000132606324258134


For the Newton method, we will try to compute the 3rd zero of the Bessel function of the first kind of order zero. This is because I wanted to try a special function for which it is not possible to compute the zero by hand. The really well approxmated answer is $x \approx 2.4048$.

In [ ]:
def foo(x):
    return sp.special.jv(0,x)

z = newton_univar(foo,3)
error = abs(z-2.4048)/2.4048
print("f(x) =",foo(z))
print("Relative error:",error)

f(x) = -1.0239231817845196e-16
Relative error: 1.062778433676799e-05


# **Discussion**

To summarize, we built some algorithms that approximate solutions for linear systems of equations: the Jacobi and Gauss-Seidel method. It was astonishing to see that for systems for which they can be used, they converge really quickly, specially the Gauss-Seidel method, its convergence rate really impressed me. 

Personally, I usually do not consider approximate methods when dealing with linear systems, since I used to think "What is the point if I can get the exact solution, up to floating-point errors"; however, after this report, I will consider these methods when dealing with huge linear systems for which the exact methods become computationally intensive. I belive that by using the Julia language, these methods could be promising for numerical computations, and really high performance. This could be done by using parallelization and generic type-stable code.

The Newton's method for roots works as good as expected, even for special functions. One thing that could be done better is the differentiation process, here we used the most basic, but there are more complicated schemes that give better approximations with a higher step-size.